In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd

import util
from util.year import year as current_year
from db import session
import model

In [15]:
evaluations = session.query(
    model.Task,
    func.count(distinct(model.User.id))
).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Task.modules).\
    join(model.Module.evaluations).\
    join(model.Evaluation.r_user)

evaluations_per_task = evaluations.\
    group_by(model.Task).order_by(model.Wave.id, model.Task.id)

evaluations_per_task[20:25]

[(Variace s opakováním, 234),
 (Variace bez opakování, 198),
 (Generování podmnožin, 181),
 (Backtracking, 148),
 (Problém osmi dam, 102)]

In [4]:
successful_evaluations = evaluations_per_task.\
    filter(model.Evaluation.ok == True)

successful_evaluations[20:25]

[(Variace s opakováním, 233),
 (Variace bez opakování, 195),
 (Generování podmnožin, 176),
 (Backtracking, 145),
 (Problém osmi dam, 98)]

In [16]:
per_module = session.query(
    model.Evaluation.user.label('user_id'),
    func.count(model.Evaluation.id).label('eval_count'),
).\
    join(model.Evaluation.r_module).join(model.Module.r_task).\
    join(model.Task.r_wave).\
    filter(model.Wave.year == current_year.id).\
    group_by(model.Evaluation.user,
             model.Evaluation.module).subquery()

EVAL_LIMITS = [10, 30, 50]
problematic_tasks = {
    limit: evaluations.
            filter(model.Evaluation.ok == False).
            join(per_module, per_module.c.user_id == model.User.id).\
            group_by(model.Task).\
            filter(per_module.c.eval_count > limit)
    for limit in EVAL_LIMITS
}

for limit, users in problematic_tasks.items():
    print(limit, users.all()[20:25])

10 [(Prostorová složitost, 67), (Pokročilí plavci, 160), (Řazení programů, 261), (Analýza četnosti slov, 55), (Debugování a testování, 113)]
50 [(Pokročilí plavci, 11), (Řazení programů, 16), (Analýza četnosti slov, 5), (Debugování a testování, 11), (PEP8, 14)]
30 [(Pokročilí plavci, 40), (Řazení programů, 56), (Analýza četnosti slov, 17), (Debugování a testování, 28), (PEP8, 39)]
